In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import time

In [34]:
workouts = pd.read_csv('workouts.csv')
workouts = workouts[workouts.user_id!=2509]
workouts = workouts[workouts.user_id!=2461]
workouts.to_csv('workouts_temp.csv')

#Manually input missing days

#workouts = pd.read_csv('workouts_temp.csv')

In [31]:
def epoch_end_convert(df):
    '''
    takes dataframe and converts date column to epoch milliseconds
    inputs:
        df = dataframe name
    '''
    try:
        return (int(time.mktime(time.strptime(df['end'], '%Y-%m-%d %H:%M:%S'))) - 14400)*1000
    except:
        #handles activities that have fractions of a second included
        return (int(time.mktime(time.strptime(df['end'][:19], '%Y-%m-%d %H:%M:%S'))) - 14400)*1000

In [32]:
#workouts_subset['end_epoch'] = workouts_subset.apply(epoch_end_convert, axis=1)
workouts['end_epoch'] = workouts.apply(epoch_end_convert, axis=1)

ValueError: ("time data '9/15/2015 22:56' does not match format '%Y-%m-%d %H:%M:%S'", u'occurred at index 0')

In [6]:
workouts.head()

,user_id,start,end,sport_id,name,intensity_score,z1,z2,z3,z4,z5,end_epoch
0,828,2015-09-15 20:08:00,2015-09-15 22:56:00,35,Track & Field,20.508564,1066,1810,1432,4668,679,1442357760000
1,828,2015-09-19 12:46:00,2015-09-19 14:09:00,35,Track & Field,16.823285,506,601,1861,696,22,1442671740000
2,828,2015-09-20 13:15:00,2015-09-20 16:00:00,35,Track & Field,20.506562,929,1302,1329,5353,321,1442764800000
3,828,2015-09-21 18:55:34.582,2015-09-21 18:58:37.876,1,Cycling,4.278678,8,170,5,0,0,1442861917000
4,828,2015-09-21 19:19:00,2015-09-21 21:12:00,35,Track & Field,19.176787,655,1757,1372,2781,28,1442869920000


#Adding in Time Periods (For Either Bikes or Races)

In [7]:
date_markers = pd.read_csv('epoch_dates.csv')

In [8]:
date_markers.head()

,ET_Date,date_start_epoch,race_period,bike_period,race_period_start,race_period_end,bike_period_start,bike_period_end
0,9/15/2015 0:00,1442289600000,1,1,1,0,1,0
1,9/16/2015 0:00,1442376000000,1,1,0,0,0,0
2,9/17/2015 0:00,1442462400000,1,1,0,0,0,0
3,9/18/2015 0:00,1442548800000,1,1,0,0,0,0
4,9/19/2015 0:00,1442635200000,1,1,0,0,0,0


In [9]:
date_markers = date_markers[date_markers['race_period_start'] > 0].reset_index()
#dictionary of race period to start epoch
race_start_dict = {}
for i in range(0, len(date_markers)):
    race_start_dict[date_markers.values[i][3]] = date_markers.values[i][2]

#date_markers = date_markers[date_markers['bike_period_start'] > 0].reset_index()
#bike_start_dict = {}
#for i in range(0, len(date_markers)):
#    bike_start_dict[date_markers.values[i][4]] = date_markers.values[i][2]


In [11]:
users = list(set(workouts.user_id.values))

In [12]:
race_start_dict

{1L: 1442289600000L,
 2L: 1443240000000L,
 3L: 1445054400000L,
 4L: 1446264000000L,
 5L: 1447473600000L}

In [13]:
#bike_start_dict

In [14]:
def add_race(df):
    if (df.end_epoch >= race_start_dict[1]) and (df.end_epoch < race_start_dict[2]):
        return 1
    elif (df.end_epoch >= race_start_dict[2]) and (df.end_epoch < race_start_dict[3]):
        return 2
    elif (df.end_epoch >= race_start_dict[3]) and (df.end_epoch < race_start_dict[4]):
        return 3
    elif (df.end_epoch >= race_start_dict[4]) and (df.end_epoch < race_start_dict[5]):
        return 4
    elif (df.end_epoch >= race_start_dict[5]) and (df.end_epoch < 1448164800000):
        return 5
    else:
        return 0


In [15]:
def add_bike(df, bike_start_dict):
    for user in users:
        if df.user_id == user:
            if (df.end_epoch <= bike_start_dict[user, 1]):
                return 1
            elif (df.end_epoch <= bike_start_dict[user, 2]) and (df.end_epoch >= bike_start_dict[user, 1]):
                return 2
            elif (df.end_epoch <= bike_start_dict[user, 3]) and (df.end_epoch >= bike_start_dict[user, 2]):
                return 3
            elif (df.end_epoch <= bike_start_dict[user, 4]) and (df.end_epoch >= bike_start_dict[user, 3]):
                return 4
            elif (df.end_epoch <= bike_start_dict[user, 5]) and (df.end_epoch >= bike_start_dict[user, 4]):
                return 5
            elif (df.end_epoch <= bike_start_dict[user, 6]) and (df.end_epoch >= bike_start_dict[user, 5]):
                return 6
            elif (df.end_epoch <= bike_start_dict[user, 7]) and (df.end_epoch >= bike_start_dict[user, 6]):
                return 7
            elif (df.end_epoch <= bike_start_dict[user, 8]) and (df.end_epoch >= bike_start_dict[user, 7]):
                return 8
            elif (df.end_epoch >= bike_start_dict[user, 8]):
                return 9
                

In [16]:
def add_periods(date_markers, df, buildup_days=0, version=None):
    if version==None:
        print "Enter Either 'race' or 'bike'!"
    if version=="race":
        #Collapse the data frame into only the start and end dates for each race period
        date_markers = date_markers[date_markers['race_period_start'] > 0].reset_index()
        #dictionary of race period to start epoch
        race_start_dict = {}
        for i in range(0, len(date_markers)):
            race_start_dict[date_markers.values[i][3]] = date_markers.values[i][2]
        #Now label each race period as 1, 2, 3, 4, or 5
        df['race_period'] = df.apply(add_race, axis=1)
        
        #add in buildup days
        df=df.sort(['user_id', 'race_period', 'end_epoch'], ascending=[True, True, False]).reset_index(drop=True)
        race_groups = df.groupby(['user_id', 'race_period']).cumcount()
        build_days = pd.DataFrame(dict(buildup_days = race_groups))
        new_df = df.join(build_days, how="left")
        return_df = new_df
    
    elif version=="bike":
        #same process, but with bike efforts
        bike_dates = pd.read_csv('huxc_bike_distances.csv')
        bike_dates['next_day'] = bike_dates['date_start_epoch'] + 86400000
        bike_start_dict={} #key is the user_id and bike period, value is the date start epoch
        for i in range(0, len(bike_dates)):
            bike_start_dict[bike_dates.values[i][0], bike_dates.values[i][1]] = bike_dates.values[i][5]
        #cycle through bike_start dict, enter placement values so day gets filled
        for user in users:
            for i in range(1, 10):
                if np.isnan(bike_start_dict[user, i]) and i == 1:
                    bike_start_dict[user, i] = bike_start_dict[2469, i]
                if np.isnan(bike_start_dict[user, i]) and i == 2:
                    bike_start_dict[user, i] = bike_start_dict[2469, i]
                if np.isnan(bike_start_dict[user, i]) and i == 3:
                    bike_start_dict[user, i] = bike_start_dict[2469, i] 
                if np.isnan(bike_start_dict[user, i]) and i == 4:
                    bike_start_dict[user, i] = bike_start_dict[2469, i]
                if np.isnan(bike_start_dict[user, i]) and i == 5:
                    bike_start_dict[user, i] = bike_start_dict[2469, i]
                if np.isnan(bike_start_dict[user, i]) and i == 6:
                    bike_start_dict[user, i] = bike_start_dict[2469, i]
                if np.isnan(bike_start_dict[user, i]) and i == 7:
                    bike_start_dict[user, i] = bike_start_dict[2469, i]
                if np.isnan(bike_start_dict[user, i]) and i == 8:
                    bike_start_dict[user, i] = bike_start_dict[2469, i]
                if np.isnan(bike_start_dict[user, i]) and i == 9:
                    bike_start_dict[user, i] = bike_start_dict[2469, i]
        df['bike_period'] = df.apply(add_bike, axis=1, args=(bike_start_dict,))
        df=df.sort(['user_id', 'bike_period', 'end_epoch'], ascending=[True, True, False]).reset_index(drop=True)
        bike_groups = df.groupby(['user_id', 'bike_period']).cumcount()
        build_days = pd.DataFrame(dict(buildup_days = bike_groups))
        new_df = df.join(build_days, how="left")
        return_df = new_df
        
    if buildup_days != 0:
        return_df = return_df[return_df['buildup_days'] < buildup_days]
        
    final_return_df = return_df[return_df['race_period']!=0]
    return return_df

In [28]:
test = add_periods(date_markers=date_markers, df=workouts, buildup_days=13, version="race")
test = test.sort(['user_id', 'end_epoch'], ascending=['True', 'True']).reset_index(drop=True)
test = test[test['race_period']!=0]


,user_id,start,end,sport_id,name,intensity_score,z1,z2,z3,z4,z5,end_epoch,race_period,buildup_days
0,828,2015-09-15 20:08:00,2015-09-15 22:56:00,35,Track & Field,20.508564,1066,1810,1432,4668,679,1442357760000,1,8
1,828,2015-09-19 12:46:00,2015-09-19 14:09:00,35,Track & Field,16.823285,506,601,1861,696,22,1442671740000,1,7
2,828,2015-09-20 13:15:00,2015-09-20 16:00:00,35,Track & Field,20.506562,929,1302,1329,5353,321,1442764800000,1,6
3,828,2015-09-21 18:55:34.582,2015-09-21 18:58:37.876,1,Cycling,4.278678,8,170,5,0,0,1442861917000,1,5
4,828,2015-09-21 19:19:00,2015-09-21 21:12:00,35,Track & Field,19.176787,655,1757,1372,2781,28,1442869920000,1,4
5,828,2015-09-22 20:31:00,2015-09-22 22:00:00,35,Track & Field,18.825382,767,992,1291,806,1323,1442959200000,1,3
6,828,2015-09-23 19:31:48.525,2015-09-23 20:21:16.705,35,Track & Field,16.790126,126,499,703,1452,89,1443039676000,1,2
7,828,2015-09-23 20:47:48.968,2015-09-23 20:58:34.201,44,Yoga,4.081088,142,13,0,0,0,1443041914000,1,1
8,828,2015-09-25 18:30:00,2015-09-25 20:47:00,0,Running,19.797324,1453,1601,1803,2406,630,1443214020000,1,0
9,828,2015-10-03 21:40:00,2015-10-03 23:23:00,0,Running,18.030997,903,2060,1612,873,56,1443914580000,2,12


In [21]:
def filter_nans(x, y):
    filtered = filter(lambda o: not np.isnan(o[0]) and not np.isnan(o[1]), zip(x, y))    
    return [el[0] for el in filtered], [el[1] for el in filtered]

In [22]:
def get_item(df_column_tuple):
    return df_column_tuple[0], df_column_tuple[1]

In [23]:
def weighted_rolling_average(data):
    #must take into account nan
    #number of things in values
    n = len(data)
    weights = []
    values = []
    for i in range(0, n):
        values.append(data[i])
        #don't care about race day
        weights.append(i+1)
    numbers = [w*v for w,v in zip(weights,values)]
    filtered_numbers, filtered_weights = filter_nans(numbers, weights)
    #print filtered_numbers, filtered_weights
    return sum(filtered_numbers)/sum(filtered_weights)

In [24]:
def weighted_rolling_average_df(df, activity_type, variable):
    weighted_avg_dict = {}
    periods = []
    if activity_type=='bike':
        periods = [1,2,3,4,5,6,7,8,9]
        for u in users:
            user_df = df[df['user_id']==u]
            for i in range(0, len(periods)):
                user_period_df=user_df[user_df['bike_period']==periods[i]]
                w_r_a = weighted_rolling_average(user_period_df[variable].reset_index(drop=True))
                weighted_avg_dict[u, periods[i], variable] = w_r_a
        
    elif activity_type=='race':
        periods = [1,2,3,4,5]
        for u in users:
            user_df = df[df['user_id']==u]
            for i in range(0, len(periods)):
                user_period_df=user_df[user_df['race_period']==periods[i]]
                w_r_a = weighted_rolling_average(user_period_df[variable].reset_index(drop=True))
                weighted_avg_dict[u, periods[i], variable] = w_r_a
                
    
    agg_df = pd.DataFrame(weighted_avg_dict.items(), columns=['temp', variable+'_temp'])
    agg_df['user_id'], agg_df[activity_type+'_period'] = zip(*agg_df['temp'].map(get_item))
    agg_df[variable] = agg_df[variable+'_temp']
    agg_df = agg_df.sort(['user_id', activity_type+'_period'], ascending = [True, True]).reset_index(drop=True)
    agg_df = agg_df.drop(['temp', variable+'_temp'], axis=1)
    return agg_df

In [25]:
test

,user_id,start,end,sport_id,name,intensity_score,z1,z2,z3,z4,z5,end_epoch,race_period,buildup_days
0,828,2015-09-15 20:08:00,2015-09-15 22:56:00,35,Track & Field,20.508564,1066,1810,1432,4668,679,1442357760000,1,8
1,828,2015-09-19 12:46:00,2015-09-19 14:09:00,35,Track & Field,16.823285,506,601,1861,696,22,1442671740000,1,7
2,828,2015-09-20 13:15:00,2015-09-20 16:00:00,35,Track & Field,20.506562,929,1302,1329,5353,321,1442764800000,1,6
3,828,2015-09-21 18:55:34.582,2015-09-21 18:58:37.876,1,Cycling,4.278678,8,170,5,0,0,1442861917000,1,5
4,828,2015-09-21 19:19:00,2015-09-21 21:12:00,35,Track & Field,19.176787,655,1757,1372,2781,28,1442869920000,1,4
5,828,2015-09-22 20:31:00,2015-09-22 22:00:00,35,Track & Field,18.825382,767,992,1291,806,1323,1442959200000,1,3
6,828,2015-09-23 19:31:48.525,2015-09-23 20:21:16.705,35,Track & Field,16.790126,126,499,703,1452,89,1443039676000,1,2
7,828,2015-09-23 20:47:48.968,2015-09-23 20:58:34.201,44,Yoga,4.081088,142,13,0,0,0,1443041914000,1,1
8,828,2015-09-25 18:30:00,2015-09-25 20:47:00,0,Running,19.797324,1453,1601,1803,2406,630,1443214020000,1,0
9,828,2015-10-03 21:40:00,2015-10-03 23:23:00,0,Running,18.030997,903,2060,1612,873,56,1443914580000,2,12


In [27]:
#new_df = weighted_rolling_average_df(test, 'race', 'intensity_score')
#new_df2 = weighted_rolling_average_df(test, 'race', 'z1')
#new_df3 = weighted_rolling_average_df(test, 'race', 'z2')
#new_df4 = weighted_rolling_average_df(test, 'race', 'z3')
#new_df5 = weighted_rolling_average_df(test, 'race', 'z4')
#new_df6 = weighted_rolling_average_df(test, 'race', 'z5')

#merge_df1 = pd.merge(new_df, new_df2, how='left')
#merge_df2 = pd.merge(merge_df1, new_df3, how='left')
#merge_df3 = pd.merge(merge_df2, new_df4, how='left')
#merge_df4 = pd.merge(merge_df3, new_df5, how='left')
#merged_df = pd.merge(merge_df4, new_df6, how='left')
